In [1]:
from pipeline_live.engine import LivePipelineEngine
from pipeline_live.data.sources.iex import list_symbols
from pipeline_live.data.alpaca.pricing import USEquityPricing
from pipeline_live.data.polygon.fundamentals import PolygonCompany
from pipeline_live.data.iex.factors import AverageDollarVolume
from zipline.pipeline import Pipeline

In [49]:
import pandas as pd

In [59]:
import pipeline_live.engine

In [48]:
help(pipeline_live.engine.LivePipelineEngine.compute_chunk)

Help on module pipeline_live.engine in pipeline_live:

NAME
    pipeline_live.engine

CLASSES
    builtins.object
        LivePipelineEngine
    
    class LivePipelineEngine(builtins.object)
     |  Methods defined here:
     |  
     |  __init__(self, list_symbols, calendar=None, populate_initial_workspace=None)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  compute_chunk(self, graph, dates, symbols, initial_workspace)
     |      Compute the Pipeline terms in the graph for the requested start and end
     |      dates.
     |      
     |      Parameters
     |      ----------
     |      graph : zipline.pipeline.graph.TermGraph
     |      dates : pd.DatetimeIndex
     |          Row labels for our root mask.
     |      symbols : list
     |          Column labels for our root mask.
     |      initial_workspace : dict
     |          Map from term -> output.
     |          Must contain at least entry for `self._root_mask_term` whose s

In [2]:
import os

In [3]:
print(os.environ["APCA_API_KEY_ID"])
print(os.environ["APCA_API_SECRET_KEY"])
print(os.environ["APCA_API_BASE_URL"])

PKTCNS2NVWO8FQA6QFB2
0e0IVetCSNw0IltXrerLVbhktvBkLQERA1BlZh2a
https://paper-api.alpaca.markets


In [5]:
l = list_symbols()

len(l)

8976

In [54]:
dates = pd.DatetimeIndex(['2020-09-01', 
                         '2020-09-02', '2020-09-03'])

dates

DatetimeIndex(['2020-09-01', '2020-09-02', '2020-09-03'], dtype='datetime64[ns]', freq=None)

In [62]:
eng.compute_chunk()

TypeError: compute_chunk() missing 4 required positional arguments: 'graph', 'dates', 'symbols', and 'initial_workspace'

In [56]:
eng = LivePipelineEngine(list_symbols, dates)
# top5 = AverageDollarVolume(window_length=20).top(5)
pipe = Pipeline({
    'close': USEquityPricing.close.latest,
    'volume': USEquityPricing.volume.latest,
})

result = eng.run_pipeline(pipe)

# df = eng.run_pipeline(pipe)

In [58]:
result

,close,volume
A,103.380000,2369888.0
AA,14.660000,11227660.0
AAAU,19.370000,1519814.0
AACG,1.130000,18730.0
AACQU,10.115000,43591.0
AADR,56.880000,8568.0
AAL,13.225000,56797603.0
AAMC,21.956400,20114.0
AAME,2.028800,2316.0
AAN,57.350000,869038.0


In [ ]:
import alpaca_trade_api as tradeapi

api = tradeapi.REST()

# Get a list of all active assets.
active_assets = api.list_assets(status='active')

# Filter the assets down to just those on NASDAQ.
nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
print(nasdaq_assets)

In [ ]:
import alpaca_trade_api as tradeapi

api = tradeapi.REST()

# Check if AAPL is tradable on the Alpaca platform.
aapl_asset = api.get_asset('RHT')
if aapl_asset.tradable:
    print('We can trade INTC.')

In [ ]:
import pandas as pd
import numpy as np

from .datamock import mock_tradeapi

from pipeline_live.data.alpaca.pricing import USEquityPricing

def test_pricing_loader(refdata, alpaca_tradeapi, data_path):
    mock_tradeapi.list_assets(alpaca_tradeapi)
    mock_tradeapi.get_barset(alpaca_tradeapi)

    loader = USEquityPricing.get_loader()
    columns = [USEquityPricing.close]
    dates = [pd.Timestamp('2018-08-22', tz='UTC')]
    symbols = ['AA']
    mask = np.zeros((1, 1), dtype='bool')
    out = loader.load_adjusted_array(columns, dates, symbols, mask)

    assert out[USEquityPricing.close]._data.shape == (1, 1)